In [63]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/")


train=pd.read_csv("numerai_training_data.csv")
test=pd.read_csv("numerai_tournament_data.csv")

In [64]:
train.head()

,t_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
0,A1,0.263669,0.711012,0.730641,0.259069,0.331150,0.300947,0.468118,0.426362,0.545662,...,0.869772,0.219606,0.690553,0.697929,0.332535,0.500681,0.418926,0.272475,0.822392,1
1,A2,0.828056,0.433518,0.435617,0.209394,0.508133,0.296114,0.082211,0.876144,0.201528,...,0.570470,0.235018,0.588986,0.767599,0.585097,0.458801,0.960031,0.732236,0.543159,0
2,A3,0.089681,0.590072,0.715500,0.921637,0.542368,0.729199,0.979151,0.009172,0.929291,...,0.847301,0.894065,0.791371,0.721615,0.202686,0.845608,0.046535,0.200791,0.654688,0
3,A4,0.796596,0.276844,0.234234,0.798410,0.129723,0.794306,0.174816,0.823313,0.209920,...,0.094406,0.817543,0.342794,0.215916,0.015114,0.207310,0.845106,0.017104,0.016130,0
4,A5,0.040664,0.036370,0.438090,0.219633,0.100569,0.097791,0.089305,0.337548,0.382942,...,0.074038,0.044325,0.844283,0.792098,0.842606,0.133044,0.002375,0.795792,0.128112,1


In [7]:
test.head(1)

,t_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21
0,8309,0.853061,0.991828,0.350143,0.654192,0.909093,0.862609,0.463147,0.911227,0.406167,...,0.925065,0.913353,0.653512,0.962262,0.95163,0.886714,0.352522,0.108802,0.945558,0.007984


In [182]:
target = 'target'
IDCol = ['t_id']
predictors=[x for x in train.columns if x not in [target]+IDCol+["predictions"]]

from sklearn import cross_validation, metrics
from datetime import datetime


def modelfit(alg,dtrain,dtest,predictors,target,IDCol,filename):
    #fit the algorithm on the data
    print datetime.now().time()
    alg.fit(dtrain[predictors],dtrain[target])
    
    #predict training data set
    dtrain["predictions"] = alg.predict_proba(dtrain[predictors])[:,1]
    
    #performs cross validation
    cv_score = cross_validation.cross_val_score(alg,dtrain[predictors],dtrain[target],cv=7,scoring='log_loss')
    
    #Print model report
    print "\nModel Report :"
    print "Train Log Loss : %.4g" % (metrics.log_loss(dtrain[target].values,dtrain["predictions"].values))
    print "CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g"%(np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))
    
    #Predict on testing data
    dtest[target]=alg.predict_proba(dtest[predictors])[:,1]
    
    #Export submission file
    IDCol.append(target)
    submission = pd.DataFrame({x : dtest[x] for x in IDCol})
    submission = submission.rename(columns={'target': 'probability'})
    if not os.path.exists("submission"):
        os.makedirs("submission")
    submission.to_csv("submission/"+filename,index=False)
    
    
    cv = pd.DataFrame(dtrain[['t_id','predictions']])
    cv = cv.rename(columns={'predictions': "CV_"+filename[:-4]})
    if not os.path.exists("CV"):
        os.makedirs("CV")
    cv.to_csv("CV/"+"CV_"+filename,index=False)
    
    print datetime.now().time()

In [178]:
# LB = 0.69915; CV = -0.6915
from sklearn.linear_model import LogisticRegression
alg3=LogisticRegression()
modelfit(alg3,train,test,predictors,target,IDCol,'alg_Logistic.csv')


12:05:10.766000

Model Report :
Train Log Loss : 0.6911
CV Score : Mean - -0.6916 | Std - 0.0004243 | Min - -0.6924 | Max - -0.6909
12:05:15.396000


In [179]:
# LB = 18.2; CV = -17.06
# Looking weird.. CV.. Need to analyze further
from sklearn.tree import DecisionTreeClassifier
alg2R=DecisionTreeClassifier(max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',random_state=111)
modelfit(alg2R,train,test,predictors,target,IDCol,'alg_DTree.csv')

12:05:21.967000

Model Report :
Train Log Loss : 0.6897
CV Score : Mean - -0.6939 | Std - 0.0006295 | Min - -0.6949 | Max - -0.6931
12:05:23.359000


In [187]:
# LB = 18.3; CV = -17.07
#####
# (max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt')
# LB = 0.69736; CV = -0.6924
#####
from sklearn.tree import ExtraTreeClassifier
algET=ExtraTreeClassifier(max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',random_state=111)
modelfit(algET,train,test,predictors,target,IDCol,'alg_ET.csv')

12:06:23.981000

Model Report :
Train Log Loss : 0.6912
CV Score : Mean - -0.6925 | Std - 0.0004172 | Min - -0.6929 | Max - -0.6918
12:06:24.844000


In [189]:
# CV = -0.8054, LB=0.79851
#####
# (n_estimators=20,max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',n_jobs=4)
# CV = -0.6914, LB=0.69810
#####
from sklearn.ensemble import RandomForestClassifier
alg1=RandomForestClassifier(n_estimators=20,max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',n_jobs=4,random_state=111)
modelfit(alg1,train,test,predictors,target,IDCol,'alg_RandomForest.csv')

12:07:36.762000

Model Report :
Train Log Loss : 0.688
CV Score : Mean - -0.6914 | Std - 0.0004759 | Min - -0.6924 | Max - -0.6907
12:07:44.333000


In [191]:
# CV = -1.792, LB = 1.68391
from sklearn.neighbors import KNeighborsClassifier
algKNN=KNeighborsClassifier(leaf_size=100,n_jobs=4,n_neighbors=20)
modelfit(algKNN,train,test,predictors,target,IDCol,'alg_KNN.csv')

12:09:53.100000

Model Report :
Train Log Loss : 0.6619
CV Score : Mean - -0.716 | Std - 0.002644 | Min - -0.7201 | Max - -0.7129
12:13:32.782000


In [192]:
# CV = -0.7017, LB = 0.72670
from sklearn.naive_bayes import GaussianNB
algGNB=GaussianNB()
modelfit(algGNB,train,test,predictors,target,IDCol,'alg_GNB.csv')

12:15:04.479000

Model Report :
Train Log Loss : 0.701
CV Score : Mean - -0.7017 | Std - 0.002806 | Min - -0.7077 | Max - -0.6987
12:15:05.514000


In [193]:
# CV = 0.6931, LB = 0.69323
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
alg3=LogisticRegression(solver='sag')#n_estimators=100,max_depth=3,min_samples_leaf=100,n_jobs=4)
ada1=AdaBoostClassifier(base_estimator=alg3)
modelfit(ada1,train,test,predictors,target,IDCol,'alg_Logistic_AdaB.csv')
print datetime.now().time()

12:15:07.900000

Model Report :
Train Log Loss : 0.6931
CV Score : Mean - -0.6931 | Std - 6.467e-06 | Min - -0.6931 | Max - -0.6931
12:31:20.749000
12:31:20.750000


In [196]:
# CV = -0.6921,  LB = 0.69875
from sklearn.svm import SVC
alg8=SVC(random_state=111)
modelfit(alg1,train,test,predictors,target,IDCol,'alg_svm.csv')

12:32:37.205000

Model Report :
Train Log Loss : 0.688
CV Score : Mean - -0.6914 | Std - 0.0004759 | Min - -0.6924 | Max - -0.6907
12:32:44.573000


In [197]:
# CV = -0.6921 LB = 0.69875
from sklearn.linear_model import SGDClassifier
alg8=SGDClassifier(loss='log')
modelfit(alg1,train,test,predictors,target,IDCol,'alg_SGD.csv')

12:33:11.432000

Model Report :
Train Log Loss : 0.688
CV Score : Mean - -0.6914 | Std - 0.0004759 | Min - -0.6924 | Max - -0.6907
12:33:18.945000


In [198]:
# CV = -0.6921 LB = 0.69875
print datetime.now().time()
from sklearn.ensemble import GradientBoostingClassifier
alg1=GradientBoostingClassifier(random_state=111)
modelfit(alg1,train,test,predictors,target,IDCol,'alg_GBM.csv')
print datetime.now().time()

12:33:26.400000
12:33:26.400000

Model Report :
Train Log Loss : 0.6828
CV Score : Mean - -0.6921 | Std - 0.0006316 | Min - -0.6934 | Max - -0.6913
12:35:56.780000
12:35:56.781000


In [211]:
# Function for xgboost
import xgboost as xgb
from datetime import datetime
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
predictors=[x for x in train.columns if x not in [target]+IDCol+["predictions"]]
def xGBmodelfit(alg,dtrain,dtest,predictors,filename,useTrainCV=True, cv_folds=5,early_stopping_rounds=200):
    print datetime.now().time()
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values,label = dtrain[target].values)
        cvresult = xgb.cv(xgb_param,xgtrain,num_boost_round=alg.get_params()['n_estimators'],
            nfold=cv_folds,metrics='logloss',early_stopping_rounds=early_stopping_rounds)#,show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])#,booster='gbtree')
    
    bestNoOfRounds = cvresult['test-logloss-mean'].argmax()
    bestAnsForThisSetting =  cvresult.ix[bestNoOfRounds]
    bestModelEval = bestAnsForThisSetting[0]
    print "Best Num of Rounds %.7g" %bestNoOfRounds
#     print "Best Ans %.7g" %bestAnsForThisSetting
    print "Best Model Eval (Test-LogLoss) %.7g" %bestModelEval
    
    alg.fit(dtrain[predictors],dtrain[target],eval_metric='logloss')#,eta=0.0001)
    
#   Predicting training data
    dtrain["predictions"] = alg.predict(dtrain[predictors])
    dtrain_predprob=alg.predict_proba(dtrain[predictors])[:,1]       
    
    print "\nModel Report"
    print "Accuracy : %.4g" %metrics.accuracy_score(dtrain[target],dtrain["predictions"].values)
    print "LogLoss Score (train) : %f" %metrics.log_loss(dtrain[target],dtrain_predprob)
    
#   Feature important plot
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar',title = 'Feature Importance')
    plt.ylabel('Feature Importance score')
    
    #Predict on testing data
    dtest[target]=alg.predict_proba(dtest[predictors])[:,1]
    
    #Export submission file
    IDCol.append(target)
    submission = pd.DataFrame({x : dtest[x] for x in IDCol})
    submission = submission.rename(columns={'target': 'probability'})
    
    if not os.path.exists("submission"):
        os.makedirs("submission")
    submission.to_csv("submission/"+filename,index=False)
    
    cv = pd.DataFrame(dtrain[['t_id','predictions']])
    cv = cv.rename(columns={'predictions': "CV_"+filename[:-4]})
    if not os.path.exists("CV"):
        os.makedirs("CV")
    cv.to_csv("CV/"+"CV_"+filename,index=False)
    
    print datetime.now().time()

In [210]:
predictors

['t_id',
 'feature1',
 'feature2',
 'feature3',
 'feature4',
 'feature5',
 'feature6',
 'feature7',
 'feature8',
 'feature9',
 'feature10',
 'feature11',
 'feature12',
 'feature13',
 'feature14',
 'feature15',
 'feature16',
 'feature17',
 'feature18',
 'feature19',
 'feature20',
 'feature21',
 'predictions']

In [213]:
# CV = 0.6929616, LB = 0.69468

xgb1 = XGBClassifier(
objective = 'binary:logistic',
learning_rate=0.05,
n_estimators = 200,
max_depth=5,
min_child_weight=5,
subsample=0.8,
colsample_bytree=0.5,
nthread=4,
seed=27)

xGBmodelfit(xgb1,train,test,predictors,'xgBoost.csv')

12:42:00.545000
Best Num of Rounds 0
Best Model Eval (Test-LogLoss) 0.6929616

Model Report
Accuracy : 0.6274
LogLoss Score (train) : 0.666240
12:42:37.086000
